# Lecture 11 - Lagrange Relaxation

Joseph Paat

*Acknowledgment:* Code taken from Lecture 1 of COMM612 by Prof. Julia Yan.
Lazy DFJ implementation adapted from the [JuMP documentation](https://jump.dev/JuMP.jl/stable/tutorials/algorithms/tsp_lazy_constraints/).

##  Header

In [ ]:
# # Dependencies: Uncomment and run this cell if you don't have these packages 
# Pkg.add("JuMP")
# Pkg.add("Gurobi")
# Pkg.add("Combinatorics")
# Pkg.add("Plots")

In [ ]:
include("tsp-src/data.jl");
include("tsp-src/opt.jl");
include("tsp-src/out.jl");

Let us first try to solve 5 TSP instances on 200 nodes with the DFJ formulation and adding lazy constraints

In [ ]:
n = 250
all_n = [i for i in 1:5]
dat = Dict()

for iter in all_n
    dat[iter] = TSPInstanceInt(n, 100, random_seed = iter)
end

In [ ]:
#dat = TSPInstanceInt(n, 100, random_seed = 1)

mdl_dfj = TSPModel(dat[1], OutputFlag =1, TimeLimit = 10)
add_dfj_callback!(mdl_dfj)
solution_dfj = solve!(mdl_dfj)

On my computer, this runs out of time.

# Comparing a heuristic to the lower bound by Held Karp

## We compare the Lin-Kernighan Heuristic to the optimal value. We look at random instances generated on 500 cities. 

In [ ]:
# Multiple random instances on 500 cities
opt_vals = Dict([("LKH",[]),("HK",[])])
    
all_n = [i for i in 1:5]

for iter in all_n
    # LK Heuristic
    @time lk_opt_tour, lk_opt_len = solve_tsp(dat[iter].d)

    #HK bound
    @time hk_bound = HK_bound(dat[iter],lk_opt_len)
    
    push!(opt_vals["LKH"], lk_opt_len)
    push!(opt_vals["HK"], hk_bound)

end

Plot results

In [ ]:
plt = Plots.plot()
for key in keys(opt_vals)
    Plots.plot!(all_n, opt_vals[key], label = key, marker = :circle)    
end

title!("Comparing LK-Heuristic to Held-Karp Lagrangian Relaxation")
xlabel!("Iteration Number")
ylabel!("Total distance traveled")
display(plt)

Plot the optimality gap

In [ ]:
rel_gap = [abs(opt_vals["LKH"][i]-opt_vals["HK"][i])/opt_vals["LKH"][i] for i in all_n]

plt2 = Plots.plot()
Plots.plot!(all_n, rel_gap, marker = :circle,legend=false)    

title!("Relative Optimality gap LK-Heuristic to Exact Solution")
xlabel!("Iteration Number")
ylabel!("Relative gap %")
display(plt2)